In [1]:
library(pacman)
p_load(tidyverse, gprofiler2, data.table)

In [4]:
# Read individual significant interaction files and put them into a single file
read_sig_inter_files <- function(fdir, levels) {
    res <- tibble()
    for (i in 0:levels) {
        fp <- file.path(fdir, paste0("level", i, "_sig_interactions.txt"))
        res <- res %>%
            bind_rows(
                read_tsv(fp, show_col_types = FALSE) %>%
                mutate(level = paste0("level", i))) %>%
                mutate(trait = if_else(grepl("Phosphatidylcholine", trait), 
                                       "Phosphatidylcholine levels*", trait),
                       trait = if_else(grepl("Phosphatidylserine", trait), 
                                       "Phosphatidylserine levels*", trait),
                       trait = if_else(grepl("Triacylglycerol", trait), 
                                       "Triacylglycerol levels*", trait),
                       trait = if_else(grepl("Cholesteryl ester", trait), 
                                       "Cholesteryl ester levels*", trait),
                       trait = if_else(grepl("Phosphatidylglycerol", trait), 
                                       "Phosphatidylglycerol levels*", trait),
                       trait = if_else(grepl("Phosphatidylethanolamine", trait),
                                       "Phosphatidylethanolamine levels*", trait),
                       trait = if_else(grepl("Phosphatidylinositol", trait), 
                                       "Phosphatidylinositol levels*", trait),
                       trait = if_else(grepl("Fatty acid", trait), 
                                       "Fatty acid levels*", trait),
                       trait = if_else(grepl("Lysophosphatidylcholine", trait),
                                       "Lysophosphatidylcholine levels*", trait),
                       trait = if_else(grepl("Lysophosphatidylethanolamine", trait),
                                       "Lysophosphatidylethanolamine levels*", trait),
                       trait = if_else(grepl("Phosphatidate", trait), 
                                       "Phosphatidate levels*", trait),
                       trait = if_else(grepl("Diacylglycerol", trait), 
                                       "Diacylglycerol levels*", trait),
                       trait = if_else(grepl("Sphingomyelin", trait), 
                                       "Sphingomyelin levels*", trait),
                       trait = if_else(grepl("Vaginal microbiome MetaCyc pathway", trait), 
                                       "Vaginal microbiome MetaCyc pathway*", trait))  %>%
        dplyr::select(-eqtl_type)
        }
    res
}

## read significant traits file (i.e., significant_enrichment_bootstrap.txt)
read_sig_enr_bootstrap_file <- function(fp) {
    traits <- read_tsv(fp, show_col_types = FALSE) %>% 
    dplyr::select(level, trait, trait_eqtls, sim_pval, adj_pval) %>% 
    arrange(level) %>% 
    distinct() %>% 
    mutate(trait = if_else(grepl("Phosphatidylcholine", trait), 
                       "Phosphatidylcholine levels*", trait),
           trait = if_else(grepl("Phosphatidylserine", trait), 
                      "Phosphatidylserine levels*", trait),
           trait = if_else(grepl("Triacylglycerol", trait), 
                      "Triacylglycerol levels*", trait),
           trait = if_else(grepl("Cholesteryl ester", trait), 
                      "Cholesteryl ester levels*", trait),
           trait = if_else(grepl("Phosphatidylglycerol", trait), 
                      "Phosphatidylglycerol levels*", trait),
           trait = if_else(grepl("Phosphatidylethanolamine", trait), 
                      "Phosphatidylethanolamine levels*", trait),
           trait = if_else(grepl("Phosphatidylinositol", trait), 
                      "Phosphatidylinositol levels*", trait),
           trait = if_else(grepl("Fatty acid", trait), 
                      "Fatty acid levels*", trait),
           trait = if_else(grepl("Lysophosphatidylcholine", trait), 
                      "Lysophosphatidylcholine levels*", trait),
           trait = if_else(grepl("Lysophosphatidylethanolamine", trait), 
                      "Lysophosphatidylethanolamine levels*", trait),
           trait = if_else(grepl("Phosphatidate", trait), 
                      "Phosphatidate levels*", trait),
           trait = if_else(grepl("Diacylglycerol", trait), 
                      "Diacylglycerol levels*", trait),
           trait = if_else(grepl("Sphingomyelin", trait), 
                      "Sphingomyelin levels*", trait),
           trait = if_else(grepl("Vaginal microbiome MetaCyc pathway", trait), 
                      "Vaginal microbiome MetaCyc pathway*", trait))
}

read_tissue_map <- function() {
    dir <- "../data/tissue_maps/Brain_Cortex/"
    res <- tibble()
    cortex_map <- file.path(list.files(dir, 
                            pattern = "significant_eqtls.txt", 
                            recursive = TRUE,
                            full.names = TRUE))
    res <- res %>%
            bind_rows(
                read_tsv(cortex_map, show_col_types = FALSE)) %>% 
            dplyr::select(gene) %>% 
            distinct()
} 

#### PD-associated network

In [ ]:
### read significant interactions file
snpi_inter <- read_sig_inter_files("../data/multimorbidity/assoc_net/", 3)

### read significant enrichment bootstrap file
snpi_traits <- read_sig_enr_bootstrap_file("../data/multimorbidity/assoc_net/significant_enrichment_bootstrap.txt")

### significant trait interactions
snp_sig_trait_inter <- snpi_inter %>%
    dplyr::select(-chromq, -posq, -rsidq, -chromt, -post, -rsidt, -corr, -dprime) %>%
    inner_join(snpi_traits %>%
               filter(sim_pval < 0.05), # retain only traits that passed sim pval < 0.05
               by = c("trait","level")) %>% 
    mutate(level = str_replace(level, "level", "L")) 

#### PD-causal network

In [ ]:
### read significant interactions file
genei_inter <- read_sig_inter_files("../data/multimorbidity/causal_net/", 3)

### read significant enrichment bootstrap file
genei_traits <- read_sig_enr_bootstrap_file("../data/multimorbidity/causal_net/significant_enrichment_bootstrap.txt")

### significant trait interactions
gene_sig_trait_inter <- genei_inter %>%
    dplyr::select(-chromq, -posq, -rsidq, -chromt, -post, -rsidt, -corr, -dprime) %>%
    inner_join(genei_traits %>%
               filter(sim_pval < 0.05), # retain only traits that passed sim pval < 0.05
               by = c("trait","level")) %>% 
    mutate(level = str_replace(level, "level", "L"))

### Biological pathway enrichment analysis using gprofiler2

In [ ]:
# Background genes
cortex_grn_genes <-  read_tissue_map()

## PD-causal network
gene_enr_res <- gost(
gene_sig_trait_inter$gene,
organism = "hsapiens",
ordered_query = FALSE,
significant = TRUE,
exclude_iea = TRUE,
evcodes = TRUE,
user_threshold = 0.05,
correction_method = "fdr",
domain_scope = "custom",
custom_bg = cortex_grn_genes$gene,
sources = c("KEGG", "REAC", "WP"))

gene_enr_res$result %>% 
dplyr::select(-query, -significant, -parents, -evidence_codes) %>% 
filter(!term_name %like% "root term" & !term_name %like% "WIKIPATHWAYS") %>%
distinct() %>% 
arrange(p_value)

## PD-associated network
snp_enr_res <- gost(
snp_sig_trait_inter_uptoL3_genes$gene,
organism = "hsapiens",
ordered_query = FALSE,
significant = TRUE,
exclude_iea = TRUE,
evcodes = TRUE,
user_threshold = 0.05,
correction_method = "fdr",
domain_scope = "custom",
custom_bg = cortex_grn_genes$gene,
sources = c("KEGG", "REAC", "WP"))
snp_enr_res$result %>% 
dplyr::select(-query, -significant, -parents, -evidence_codes) %>% 
filter(!term_name %like% "root term" & !term_name %like% "WIKIPATHWAYS") %>%
distinct() %>% 
arrange(p_value)

In [ ]:
cortex_grn_genes <-  read_tissue_map()
suggestive_causal <- read_tsv("../data/bcgrn_MR/cortex_grn_singlesnp_sugg.txt", 
                             show_col_types = FALSE) %>% 
    distinct(exposure)
sugg_causal_enr_res <- gost(
suggestive_causal$exposure,
organism = "hsapiens",
ordered_query = FALSE,
significant = TRUE,
exclude_iea = TRUE,
evcodes = TRUE,
user_threshold = 0.05,
correction_method = "fdr",
domain_scope = "custom",
custom_bg = cortex_grn_genes$gene,
sources = c("KEGG", "REAC", "WP"))
snp_enr_res$result %>% 
dplyr::select(-query, -significant, -parents, -evidence_codes) %>% 
filter(!term_name %like% "root term" & !term_name %like% "WIKIPATHWAYS") %>%
distinct() %>% 
arrange(p_value) 